In [1]:
import os
import pickle
import scipy.io
import numpy as np
import numpy.linalg


from scfw.scopt import scopt
from scfw.prox_grad import prox_grad
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
data_folder = '../data'

problem_name = 'syn_1000_800_10_50'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [3]:
results_folder = './results/'
results_file=os.path.join(results_folder,'portfolio',problem_name+'.pckl')
results = {problem_name: {}}
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results=pickle.load(f)
        except Exception: # so many things could go wrong, can't be more specific.
            pass 

In [4]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
f0,dot_product0=pr.portfolio(R,x0)
sigma_f=min(np.linalg.eigvalsh(pr.hess_portfolio(R, x0, dot_product0)))
if sigma_f<0:
    sigma_f=1e-10
terminate_tol = 1e-15

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
    'rho':np.sqrt(n), #parameters for ll00
    'diam_X':np.sqrt(2),
    'sigma_f':sigma_f,                   
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [6]:
func_x = lambda x: pr.portfolio(R,x)
func_beta = lambda x, s, beta, dot_product, dot_product_s:lr.log_reg(R, (1 - beta) * x + beta * s, (dot_product)*(1-beta)+(dot_product_s)*beta,gamma)
func_beta = lambda x, s, beta, dot_product, dot_product_s: pr.portfolio(R,(1 - beta) * x + beta * s,(1 - beta) * dot_product + beta * dot_product_s)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
prox_func = lambda x, L: pr.proj_simplex(x)

## Prox grad

In [7]:
def run_prox_grad(problem_name):
    data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
    R = data['W']
    results = {problem_name: {}}
    results_file = os.path.join(results_folder,'portfolio', problem_name + '.pckl')
    results = {problem_name: {}}
    if os.path.isfile(results_file):
        with open(results_file, "rb") as f:
                results=pickle.load(f)
    N, n = R.shape
    Mf = 2
    nu = 3
    I = np.eye(n)
    Iv = np.ones(n)
    #running parameters
    x0 = np.ones(n) / n
    terminate_tol = 1e-15
    prox_params={
        #parameters for SCOPT
        'iter_prox': 1000,
        'Lest': 'estimate',#,'estimate', #estimate L
        'bb_type': 2,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
        'btk_iters': 100,
        'backtracking': True
    }
    func_x = lambda x: pr.portfolio(R,x)
    func_beta = lambda x, s, beta, dot_product, dot_product_s:lr.log_reg(R, (1 - beta) * x + beta * s, (dot_product)*(1-beta)+(dot_product_s)*beta,gamma)
    func_beta = lambda x, s, beta, dot_product, dot_product_s: pr.portfolio(R,(1 - beta) * x + beta * s,(1 - beta) * dot_product + beta * dot_product_s)
    grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
    grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
    hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
    hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
    hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
    extra_func = lambda x: R @ x
    linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
    llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
    prox_func = lambda x, L: pr.proj_simplex(x)
    
    x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
            grad_x,
            hess_mult_vec_x,
            prox_func,
            Mf,
            x0,
            prox_params,
            eps=terminate_tol,
            print_every=500)
    results[problem_name]['prox_grad'] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)  

In [8]:
data_list = ['syn_1000_800_10_50', 'syn_1000_1200_10_50', 'syn_1000_1500_10_50',
             'syn_1000_800_10_50_1', 'syn_1000_1200_10_50_1', 'syn_1000_1500_10_50_1',
            'syn_1000_800_10_50_2', 'syn_1000_1200_10_50_2', 'syn_1000_1500_10_50_2',
            'syn_1000_800_10_50_3', 'syn_1000_1200_10_50_3', 'syn_1000_1500_10_50_3']
data_list = ['syn_1000_800_10_50', 'syn_1000_1200_10_50']
for problem_name in data_list:
    run_prox_grad(problem_name)

iter =    1, stepsize = 2.150e-02, rdiff = 1.160e-04 , f = 0.0563501
iter =  500, stepsize = 1.000e+00, rdiff = 5.601e-12 , f = -4.57141
iter = 1000, stepsize = 1.000e+00, rdiff = 8.500e-14 , f = -4.57141
7.258364915847778
iter =    1, stepsize = 1.867e-02, rdiff = 8.910e-05 , f = 0.0803035
iter =  500, stepsize = 1.000e+00, rdiff = 6.744e-13 , f = -5.11902
iter = 1000, stepsize = 1.000e+00, rdiff = 1.408e-13 , f = -5.11902
14.112051725387573


## Run FW

In [9]:
run_alpha_policies = ['backtracking','sc','sc_backtracking','sc_hybrid']

for policy in run_alpha_policies:
    print(policy)
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x, 
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    
#with open(results_file, 'wb') as f:
#        pickle.dump(results, f)

backtracking
********* Algorithm starts *********
iter=1, stepsize=1.00e+00, criterion=1.00e-05, upper_bound=0.05635009446772449, lower_bound=-5.176361103704231, real_Gap=5.23e+00, f_val=0.05635009446772449
iter=1000, stepsize=5.74e-08, criterion=3.59e-10, upper_bound=-4.571413550381484, lower_bound=-4.571413558395279, real_Gap=8.01e-09, f_val=-4.571413550381164
iter=2000, stepsize=7.53e-08, criterion=1.36e-11, upper_bound=-4.571413550381705, lower_bound=-4.571413556802577, real_Gap=6.42e-09, f_val=-4.5714135503814575
iter=3000, stepsize=2.50e-07, criterion=5.54e-12, upper_bound=-4.571413550382122, lower_bound=-4.5714135548766555, real_Gap=4.49e-09, f_val=-4.5714135503820446
iter=4000, stepsize=3.77e-08, criterion=5.54e-12, upper_bound=-4.571413550382257, lower_bound=-4.571413553696182, real_Gap=3.31e-09, f_val=-4.571413550382046
iter=5000, stepsize=4.04e-08, criterion=5.54e-12, upper_bound=-4.5714135503832996, lower_bound=-4.571413553696182, real_Gap=3.31e-09, f_val=-4.571413550382898

In [9]:
run_alpha_policies = []#["backtracking","standard","line_search","icml"]

for policy in run_alpha_policies:
    print(policy)
    #policy_results = []
    #for i in range(len(points)):
    #    x0 = points[i]
    x, alpha_hist, Gap_hist, Q_hist, time_hist = frank_wolfe(func_x,
                        func_beta,                                       
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,                                       
                        alpha_policy=policy,                                                    
                        eps=terminate_tol,
                        print_every=1000, 
                        debug_info=False)
        
    results[problem_name][policy]={
    'x': x,
    'alpha_hist': alpha_hist,        
    'Q_hist': Q_hist,
    'Gap_hist': Gap_hist,        
    'time_hist': time_hist,}
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
#    results[problem_name][policy] = policy_results

## Run SCOPT

In [11]:
x, alpha_hist, Q_hist, time_hist = scopt(func_x,
        grad_x,
        hess_mult_x,
        hess_mult_vec_x,
        Mf,
        nu,
        prox_func,
        x0,  
        sc_params,                                              
        eps=terminate_tol,                                              
        print_every=1)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
#    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
#    'grad_hist': grad_hist
}

Fista err = 9.333e-06; Subiter =  40; subproblem converged!
iter =    1, stepsize = 5.975e-01, rdiff = 4.253e-01 , f = 0.0689425
Fista err = 9.845e-06; Subiter =  29; subproblem converged!
iter =    2, stepsize = 7.868e-01, rdiff = 1.723e-01 , f = -2.36287
Fista err = 9.975e-06; Subiter =  91; subproblem converged!
iter =    3, stepsize = 9.442e-01, rdiff = 3.773e-02 , f = -3.587
Fista err = 9.311e-06; Subiter =  13; subproblem converged!
iter =    4, stepsize = 9.969e-01, rdiff = 1.956e-03 , f = -3.89293
Fista err = 6.277e-06; Subiter =   3; subproblem converged!
iter =    5, stepsize = 9.999e-01, rdiff = 4.274e-05 , f = -3.91085
Fista err = 6.668e-12; Subiter =   2; subproblem converged!
boo
Convergence achieved!
iter =    6, stepsize = 0.000e+00, rdiff = 0.000e+00,value=-3.91091
1.5409066677093506


KeyError: 'syn_1000_800_10_50'